<a href="https://colab.research.google.com/github/JM3927/Hand-ASL-Estimation/blob/main/AI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# https://www.youtube.com/watch?v=a99p_fAr6e4
!git clone https://github.com/kinivi/hand-gesture-recognition-mediapipe.git

Cloning into 'hand-gesture-recognition-mediapipe'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 153 (delta 50), reused 45 (delta 45), pack-reused 83 (from 1)
Receiving objects: 100% (153/153), 1.82 MiB | 8.08 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [5]:
!pip install opencv-python
!pip install mediapipe
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled pr

In [14]:
# Get model from absolute path
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/latest/hand_landmarker.task

Importing Images - TRIM AMOUNT TO 10-20 PER FOLDER

In [15]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'asl-alphabet' dataset.
Path to dataset files: /kaggle/input/asl-alphabet


Image Scaling Via Pillow

In [16]:
# !pip install Pillow
from PIL import Image
import os

def resize_image(image_path, output_folder, factor):
    try:
        with Image.open(image_path) as img:
            new_size=(int(img.size[0]*factor),int(img.size[1]* factor))
            output_path=os.path.join(output_folder, os.path.basename(image_path))
            img = img.resize(new_size)
            img.save(output_path)
            print(f"Resized image of {new_size} saved to {output_path}")
    except Exception as e:
        print(f"Error resizing image{image_path}: {e}")

# def create_output_directory(path):
#     if not os.path.exists(path):
#         os.makedirs(path)

# def get_output_directory(input_directory):
#     relative_path = os.path.relpath(input_directory, input_dir)
#     return os.path.join(output_dir, relative_path)
# for subdir, dirs, files in os.walk(input_dir):
#     for file in files:
#         if file.endswith(".JPG"):
#             input_file_path = os.path.join(subdir, file)
#             output_directory=get_output_directory(subdir)
#             create_output_directory(output_directory)
#             output_file_path=os.path.join(output_directory, file)
#             if not os.path.exists(output_file_path):
#                 resize_image(input_file_path, output_directory, 0.5)

def grab_first_20_images(root):
    results = []

    for mainDir, subDirs, files in os.walk(root):
        # Take only the first 20 files in this folder
        files = sorted(files)
        top20 = files[:4]

        # Build full paths
        for fname in top20:
            results.append(os.path.join(mainDir, fname))

    return results

ASL_Path = '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'

# Creates unordered list of file paths
subdirList = grab_first_20_images(ASL_Path)
print(subdirList)

resizedImagesPath = "/content/resized_images"
os.makedirs(resizedImagesPath, exist_ok=True)

for path in subdirList:
    resize_image(path, resizedImagesPath, 1.28)

['/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/N/N1.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/N/N10.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/N/N100.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/N/N1000.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/R/R1.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/R/R10.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/R/R100.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/R/R1000.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/space/space1.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/space/space10.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/space/space100.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/space/space1000.jpg', '/kaggle/input/asl-alphab

Running Through Base Model for Hand Pose Estimation

In [36]:
import csv
import mediapipe as mp

# List of letter and the list of coordinates for each image
def grab_all_files(root):
    results = []

    for mainDir, subDirs, files in os.walk(root):
        files = sorted(files)

        # Build full paths
        for fname in files:
            results.append(os.path.join(mainDir, fname))

    return results

imagesPath = '/content/resized_images'
fileList = grab_all_files(imagesPath)
print(len(fileList))
print(fileList)

# ex path: /content/resized_images/A1.jpg
# Wanted Output: A
folderNames = []

for path in fileList:
    ex = path.split('/')[-1]
    ex = ex.split('1')[0]
    # List of all group names
    if ex not in folderNames:
        folderNames.append(ex)

# Save data as .csv


116
['/content/resized_images/A1.jpg', '/content/resized_images/A10.jpg', '/content/resized_images/A100.jpg', '/content/resized_images/A1000.jpg', '/content/resized_images/B1.jpg', '/content/resized_images/B10.jpg', '/content/resized_images/B100.jpg', '/content/resized_images/B1000.jpg', '/content/resized_images/C1.jpg', '/content/resized_images/C10.jpg', '/content/resized_images/C100.jpg', '/content/resized_images/C1000.jpg', '/content/resized_images/D1.jpg', '/content/resized_images/D10.jpg', '/content/resized_images/D100.jpg', '/content/resized_images/D1000.jpg', '/content/resized_images/E1.jpg', '/content/resized_images/E10.jpg', '/content/resized_images/E100.jpg', '/content/resized_images/E1000.jpg', '/content/resized_images/F1.jpg', '/content/resized_images/F10.jpg', '/content/resized_images/F100.jpg', '/content/resized_images/F1000.jpg', '/content/resized_images/G1.jpg', '/content/resized_images/G10.jpg', '/content/resized_images/G100.jpg', '/content/resized_images/G1000.jpg', '

In [45]:
# Passing file list through base model to obtain (mp needs python3.12 or earlier)
import cv2
from google.colab.patches import cv2_imshow
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Test image
# !wget -q -O image.jpg https://storage.googleapis.com/mediapipe-tasks/hand_landmarker/woman_hands.jpg

base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=1)
detector = vision.HandLandmarker.create_from_options(options)

csvList = [][]
newFileList = []
x = []
y = []

numDeleted = 0;

# for i in range(len(fileList)):
for i in range(1):
    imagePath = fileList[i]
    image = mp.Image.create_from_file('/content/image.jpg')

    detection_result = detector.detect(image)
    print(detection_result)

    # Iterate through detection_result.hand_landmarks and extract x, y coordinates
    if detection_result.hand_landmarks:
        newFileList.append(imagePath)
        for hand_landmarks in detection_result.hand_landmarks:
            csvList[i].append(newFileList[i])
            # Append all 21 keypoint values
            for landmark in hand_landmarks:
                csvList[i].append(landmark.x)
                csvList[i].append(landmark.y)
    # else:
    #     numDeleted += 1



print(len(newFileList))
print(newFileList)
print(len(csvList))
print(csvList)


SyntaxError: invalid syntax (ipython-input-2769239237.py, line 15)

Running Re-Training for mini NN

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.keras'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

NUM_CLASSES = 4

# Normalized on 0, prev. is normalized on input width/height
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)


model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

# Loading the saved model
model = tf.keras.models.load_model(model_save_path)
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_8 (Dropout)             │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 4)              │            44 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,114 (4.35 KB)

 Trainable params: 1,114 (4.35 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
24/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3419 - loss: 1.3949
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.3449 - loss: 1.3874 - val_accuracy: 0.4261 - val_loss: 1.2495
Epoch 2/1000
24/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3917 - loss: 1.2619 
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3933 - loss: 1.2593 - val_accuracy: 0.5079 - val_loss: 1.1595
Epoch 3/1000
24/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4226 - loss: 1.1863 
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4232 - loss: 1.1860 - val_accuracy: 0.5789 - val_loss: 1.0829
Epoch 4/1000
24/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4732 - loss: 1.1372 
Epoch 4: saving model to model/keypoint_classifier/keypoint_classifier.keras

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
